# New csv Import

In [290]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [291]:
df=pd.read_csv('cleaned.csv')
print(df.shape)

(1025272, 19)


In [292]:
df.head()

,Unnamed: 0,cancel,year,zip.code,house.color,ni.age,len.at.res,credit,coverage.type,dwelling.type,premium,sales.channel,ni.gender,ni.marital.status,n.adults,n.children,tenure,claim.ind,state
0,0,0,2013,98004.0,blue,37.0,18.000000,high,C,Tenant,950.507336,Broker,F,0.0,2.0,0.0,15.0,0.0,WA
1,1,0,2013,85017.0,white,40.0,17.000000,medium,A,Condo,909.346046,Broker,F,1.0,5.0,0.0,15.0,0.0,AZ
2,2,0,2013,80021.0,white,45.0,14.000000,high,A,House,897.084502,Phone,M,0.0,1.0,0.0,14.0,0.0,CO
3,3,0,2013,85002.0,red,45.0,24.366136,high,A,House,979.039007,Phone,M,0.0,5.0,0.0,22.0,1.0,AZ
4,4,0,2013,50001.0,yellow,36.0,16.000000,high,C,House,932.379027,Broker,M,1.0,2.0,4.0,4.0,0.0,IA


# Compute Dummy Variables

In [293]:
#one-hot编码: one-hot编码是一种对离散特征值的编码方式，在LR模型中常用到，用于给线性模型增加非线性能力。

#from sklearn import preprocessing

#data = [[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]]
#encoder = preprocessing.OneHotEncoder().fit(data)
#encoder.transform(data).toarray()

In [294]:
df.dtypes

Unnamed: 0             int64
cancel                 int64
year                   int64
zip.code             float64
house.color           object
ni.age               float64
len.at.res           float64
credit                object
coverage.type         object
dwelling.type         object
premium              float64
sales.channel         object
ni.gender             object
ni.marital.status    float64
n.adults             float64
n.children           float64
tenure               float64
claim.ind            float64
state                 object
dtype: object

In [295]:
# continue表示跳过后面的程序，重新循环，而pass表示站位，后面的代码（else之前）还是会执行

#for x in df.columns:
#    if df[x].dtypes == 'object':
#        print(x) 
#    else:
#        continue

In [296]:
objectList = []
for x in df.columns:
    if df[x].dtypes == 'object':
        objectList.append(x)
    else:
        continue

print(objectList)

['house.color', 'credit', 'coverage.type', 'dwelling.type', 'sales.channel', 'ni.gender', 'state']


In [297]:
dummyMatrix = pd.get_dummies(df[objectList])

In [298]:
dummyMatrix.head()

,house.color_blue,house.color_red,house.color_white,house.color_yellow,credit_high,credit_low,credit_medium,coverage.type_A,coverage.type_B,coverage.type_C,...,sales.channel_Online,sales.channel_Phone,ni.gender_F,ni.gender_M,state_AZ,state_CO,state_IA,state_PA,state_VA,state_WA
0,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
1,0,0,1,0,0,0,1,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,1,0,1,0,0,1,0,0,...,0,1,0,1,0,1,0,0,0,0
3,0,1,0,0,1,0,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0
4,0,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0


In [299]:
# quality check of 'dummyMatrix'
print(np.sum(dummyMatrix, axis=1).unique())
print(dummyMatrix.shape)

[7]
(1025272, 24)


In [300]:
# creat new df
df = df.drop(objectList, 1)
df = df.drop('Unnamed: 0', 1)
df = df.drop('zip.code', 1)
df = pd.concat([df, dummyMatrix], axis = 1)

In [301]:
df.shape

(1025272, 34)

In [302]:
#df.head().transpose()

In [303]:
df = df.drop('year', 1)

In [304]:
#https://github.com/scikit-learn-contrib/imbalanced-learn/blob/master/README.rst

# https://elitedatascience.com/imbalanced-classes
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df.cancel==0]
df_minority = df[df.cancel==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=776729,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.cancel.value_counts()
# 1    776729
# 0    776729
# Name: balance, dtype: int64

#print(df_upsampled.head())
df = df_upsampled

# Standardize Numerical Variable

In [305]:
from sklearn.model_selection import train_test_split
X = df.drop('cancel', 1)
y = df.cancel
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=31)

In [306]:
from sklearn.preprocessing import StandardScaler 

col_names = ['ni.age', 'len.at.res', 'premium','tenure']###,'n.adults', 'n.children']
scaler = StandardScaler().fit(X_train_scaled[col_names].values)

In [307]:
X_train_scaled = X_train.copy()
X_train_scaled[col_names]= scaler.transform(X_train_scaled[col_names].values)
#X_train_scaled.head()

In [308]:
X_valid_scaled = X_valid.copy()
X_valid_scaled[col_names]= scaler.transform(X_valid_scaled[col_names].values)
#X_valid_scaled.head()

# Neural Network

http://sdsawtelle.github.io/blog/output/week4-andrew-ng-machine-learning-with-python.html
https://stackoverflow.com/questions/46661373/neural-network-gets-stuck


In [312]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(33,65,2), 
                    activation='logistic',
                    solver='adam', 
                    alpha=0.0001, 
                    batch_size='auto', 
                    learning_rate='constant', 
                    learning_rate_init=0.001, 
                    power_t=0.5, 
                    max_iter=200, 
                    shuffle=True, 
                    random_state=None, 
                    tol=0.0001, 
                    verbose=True, 
                    warm_start=False, 
                    momentum=0.9, 
                    nesterovs_momentum=True, 
                    early_stopping=True, 
                    validation_fraction=0.1, 
                    beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)

In [313]:
import time
from datetime import datetime

start_time = datetime.now()

####################
# do your work here
NN = mlp.fit(X_train_scaled,y_train)
#
####################

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Iteration 1, loss = 0.62013773
Validation score: 0.665320
Iteration 2, loss = 0.61087286
Validation score: 0.665304
Iteration 3, loss = 0.60986543
Validation score: 0.664982
Iteration 4, loss = 0.60922447
Validation score: 0.666125
Iteration 5, loss = 0.60893167
Validation score: 0.665811
Iteration 6, loss = 0.60871167
Validation score: 0.665280
Iteration 7, loss = 0.60857328
Validation score: 0.666592
Iteration 8, loss = 0.60848144
Validation score: 0.665723
Iteration 9, loss = 0.60837835
Validation score: 0.666246
Iteration 10, loss = 0.60830077
Validation score: 0.666495
Iteration 11, loss = 0.60820240
Validation score: 0.666648
Iteration 12, loss = 0.60813308
Validation score: 0.666471
Iteration 13, loss = 0.60811350
Validation score: 0.665658
Iteration 14, loss = 0.60801938
Validation score: 0.666777
Iteration 15, loss = 0.60800705
Validation score: 0.665650
Iteration 16, loss = 0.60792332
Validation score: 0.666077
Iteration 17, loss = 0.60787918
Validation score: 0.666632
Iterat

In [314]:
pred = NN.predict(X_train_scaled)
y = y_train
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Train.AUC: {}'.format(metrics.auc(fpr, tpr)))

pred = NN.predict(X_valid_scaled)
y = y_valid
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Valid.AUC: {}'.format(metrics.auc(fpr, tpr)))

Train.AUC: 0.6685468917288245
Valid.AUC: 0.6666719278048123


In [315]:
from collections import Counter
Counter(pred)

Counter({1: 150745, 0: 159947})

In [316]:
from collections import Counter
Counter(y_valid)

Counter({1: 155197, 0: 155495})

In [317]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_valid, pred)

array([[105942,  49553],
       [ 54005, 101192]])

![title](Confusion_Matrix.png)

# XGboost

In [318]:
# Just in case, your jupyter shuts down, try following:

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [319]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_validate#Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn import metrics  

#from xgboost import XGBClassifier

dtrain = xgb.DMatrix(X_train, y_train)  # 使用XGBoost的原生版本需要对数据进行转化
dvalid = xgb.DMatrix(X_valid, y_valid)

In [320]:
param = {'max_depth':5, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)

In [321]:
pred = bst.predict(dvalid)
predictions = [round(value) for value in pred]

In [322]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_valid, predictions)

array([[106104,  49391],
       [ 57479,  97718]])

In [323]:
fpr, tpr, thresholds = metrics.roc_curve(y_valid, predictions, pos_label=1)
#metrics.auc(fpr, tpr)
print('Valid.AUC: {}'.format(metrics.auc(fpr, tpr)))

Valid.AUC: 0.6560006182222351


In [223]:
def xgb_model(train_data, train_label, test_data, test_label):
    clf = xgb.XGBClassifier(max_depth=5,
                           min_child_weight=1,
                           learning_rate=0.1,
                           n_estimators=10,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           seed=1,
                           missing=None)
    clf.fit(train_data, train_label, eval_metric='auc', verbose=True,
            eval_set=[(test_data, test_label)], early_stopping_rounds=100)
    y_pre = clf.predict(test_data)
    y_pro = clf.predict_proba(test_data)[:, 1]
    #print "AUC Score : %f" % metrics.roc_auc_score(test_label, y_pro)
    #print"Accuracy : %.4g" % metrics.accuracy_score(test_label, y_pre)
    return clf 

In [224]:
xgb_model(X_train, y_train, X_valid, y_valid)

[0]	validation_0-auc:0.694957
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.695444
[2]	validation_0-auc:0.696755
[3]	validation_0-auc:0.697967
[4]	validation_0-auc:0.699602
[5]	validation_0-auc:0.69997
[6]	validation_0-auc:0.701123
[7]	validation_0-auc:0.703594
[8]	validation_0-auc:0.70712
[9]	validation_0-auc:0.708534


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=0, scale_pos_weight=1, seed=1, silent=True,
       subsample=1)

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [103]:
### https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [106]:
predictors = [x for x in df.columns]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X_train, predictors)

KeyError: "['cancel'] not in index"

In [93]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 10


print('running cross validation')
# do cross validation, this will print result out as
# [iteration]  metric_name:mean_value+std_value
# std_value is standard deviation of the metric
xgb.cv(param, dtrain, num_round, nfold=5,
       metrics={'auc'}, seed=0,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])

running cross validation
[0]	train-auc:0.653408+0.000447852	test-auc:0.653407+0.00178609
[1]	train-auc:0.685495+0.000491922	test-auc:0.68549+0.00196262
[2]	train-auc:0.702025+0.000487052	test-auc:0.702005+0.00202885
[3]	train-auc:0.710894+0.00052503	test-auc:0.710872+0.00219602
[4]	train-auc:0.713783+0.000515608	test-auc:0.713735+0.00213019
[5]	train-auc:0.716839+0.000445787	test-auc:0.716725+0.00250291
[6]	train-auc:0.718342+0.000505309	test-auc:0.718169+0.00267359
[7]	train-auc:0.7195+0.000551819	test-auc:0.719116+0.00267701
[8]	train-auc:0.720621+0.00050183	test-auc:0.720199+0.00267897
[9]	train-auc:0.721782+0.000370634	test-auc:0.721434+0.00222737


,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.653408,0.000448,0.653407,0.001786
1,0.685495,0.000492,0.685490,0.001963
2,0.702025,0.000487,0.702005,0.002029
3,0.710894,0.000525,0.710872,0.002196
4,0.713783,0.000516,0.713735,0.002130
5,0.716839,0.000446,0.716725,0.002503
6,0.718342,0.000505,0.718169,0.002674
7,0.719500,0.000552,0.719116,0.002677
8,0.720621,0.000502,0.720199,0.002679
9,0.721782,0.000371,0.721434,0.002227


In [71]:
cv_params = {'n_estimators': [6,10,20,50],'learning_rate':[0.1,0.01,0.05]}
other_params = { 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'silent':0,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}


model = xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=80)


# Support Vector Machine

##### Warning Normalizing data:
For many estimators, including the SVMs, having datasets with unit standard deviation for each feature is important to get good prediction.
https://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html

### Linear SVM

In [152]:
import time
from datetime import datetime

start_time = datetime.now()

####################
# do your work here
from sklearn.svm import LinearSVC 
linear_svm = LinearSVC(max_iter=1000).fit(X_train_scaled, y_train)


#from sklearn import linear_model
#clf = linear_model.SGDClassifier(max_iter=10000, tol=1e-3)
#linear_svm = clf.fit(X_train, y_train)

#
####################

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:05:09.441819


/anaconda3/envs/py_3_6/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [153]:
pred = linear_svm.predict(X_train_scaled)
y = y_train
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Train.AUC: {}'.format(metrics.auc(fpr, tpr)))

pred = linear_svm.predict(X_valid_scaled)
y = y_valid
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Valid.AUC: {}'.format(metrics.auc(fpr, tpr)))

Train.AUC: 0.5653396698996822
Valid.AUC: 0.5646395878142705


### RBF

In [149]:
from sklearn.svm import SVC  
SVM_rbf = SVC(C=0.001, 
              kernel='rbf', 
              degree=3, 
              gamma='auto',
              coef0=0.0, 
              shrinking=True, 
              probability=False, 
              tol=0.001, 
              cache_size=200, 
              class_weight=None, 
              verbose=False, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
SVM_rbf_clf = SVM_rbf.fit(X_train_scaled[1:10000], y_train[1:10000]) 

In [151]:
pred = SVM_rbf_clf.predict(X_train_scaled[1:10000])
y = y_train[1:10000]
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Train.AUC: {}'.format(metrics.auc(fpr, tpr)))

pred = SVM_rbf_clf.predict(X_valid_scaled[1:2000])
y = y_valid[1:2000]
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Valid.AUC: {}'.format(metrics.auc(fpr, tpr)))

Train.AUC: 0.5
Valid.AUC: 0.5


### poly kernel

In [26]:
import time
from datetime import datetime

start_time = datetime.now()

####################
# do your work here
from sklearn.svm import SVC  
svclassifier = SVC(kernel='poly', gamma = 'auto')  
model2 = svclassifier.fit(X_train[1:10000], y_train[1:10000]) 

#
####################

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:05.615909


In [27]:
pred = model2.predict(X_train[1:10000])
y = y_train[1:10000]
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Train.AUC: {}'.format(metrics.auc(fpr, tpr)))

pred = model2.predict(X_valid[1:10000])
y = y_valid[1:10000]
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Valid.AUC: {}'.format(metrics.auc(fpr, tpr)))

Train.AUC: 0.5084162940473365
Valid.AUC: 0.5089680078592158


### sigmoid

In [28]:
import time
from datetime import datetime

start_time = datetime.now()

####################
# do your work here
from sklearn.svm import SVC  
svclassifier = SVC(kernel='sigmoid', gamma = 'auto')  
model2 = svclassifier.fit(X_train[1:10000], y_train[1:10000]) 

#
####################

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:05.384301


In [29]:
pred = model2.predict(X_train[1:10000])
y = y_train[1:10000]
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Train.AUC: {}'.format(metrics.auc(fpr, tpr)))

pred = model2.predict(X_valid[1:10000])
y = y_valid[1:10000]
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Valid.AUC: {}'.format(metrics.auc(fpr, tpr)))

Train.AUC: 0.5439117072122638
Valid.AUC: 0.5461501001213735


In [30]:
import time
from datetime import datetime

start_time = datetime.now()

####################
# do your work here
from sklearn.svm import NuSVC
clf = NuSVC(gamma='scale')
model3 = clf.fit(X_train[1:10000], y_train[1:10000]) 
#
####################

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:06.528583


In [31]:
pred = model3.predict(X_train[1:10000])
y = y_train[1:10000]
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Train.AUC: {}'.format(metrics.auc(fpr, tpr)))

pred = model3.predict(X_valid[1:10000])
y = y_valid[1:10000]
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
#metrics.auc(fpr, tpr)
print('Valid.AUC: {}'.format(metrics.auc(fpr, tpr)))

Train.AUC: 0.5285687282130782
Valid.AUC: 0.5215246095217647


In [166]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [175]:
import time
from datetime import datetime

start_time = datetime.now()

####################
# do your work here
best_parm = svc_param_selection(X_train[1:1000], y_train[1:1000], 50)
#
####################

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(best_parm)

Duration: 0:01:28.578195
{'C': 10, 'gamma': 0.01}


/anaconda3/envs/py_3_6/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [176]:
from sklearn.svm import SVC
svc = SVC(C =10, gamma = 0.01, kernel='rbf')
svc.fit(X_train[1:1000], y_train[1:1000])

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [177]:
# error rate
pred = svc.predict(X_valid[1:1000])
y = y_valid[1:1000]


In [178]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.5024562771527236